In [ ]:
# Installing required packages
!pip install pyspark
!pip install findspark

In [3]:
# starting spark
import findspark
findspark.init()

In [4]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the spark context. 
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, lit, udf, length, substring, expr, regexp_replace, sum_distinct
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType
from pyspark.sql import functions as F

In [5]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = (SparkSession.builder
                     .master("local[7]")
                     .appName("Aceleração PySpark - Capgemini"))

In [6]:
schema_airports = StructType([
    StructField("faa",  StringType(),  True),
    StructField("name", StringType(),  True),
    StructField("lat",  FloatType(),   True),
    StructField("lon",  FloatType(),   True),
    StructField("alt",  IntegerType(), True),
    StructField("tz",   IntegerType(), True),
    StructField("dst",  StringType(),  True)
])

schema_planes = StructType([
    StructField("tailnum",      StringType(),  True),
    StructField("year",         IntegerType(), True),
    StructField("type",         StringType(),  True),
    StructField("manufacturer", StringType(),  True),
    StructField("model",        StringType(),  True),
    StructField("engines",      IntegerType(), True),
    StructField("seats",        IntegerType(), True),
    StructField("speed",        IntegerType(), True),
    StructField("engine",       StringType(),  True)
])

schema_flights = StructType([
    StructField("year",      IntegerType(), True),
    StructField("month",     IntegerType(), True),
    StructField("day",       IntegerType(), True),
    StructField("dep_time",  StringType(),  True),
    StructField("dep_delay", IntegerType(), True),
    StructField("arr_time",  StringType(),  True),
    StructField("arr_delay", IntegerType(), True),
    StructField("carrier",   StringType(),  True),
    StructField("tailnum",   StringType(),  True),
    StructField("flight",    StringType(),  True),
    StructField("origin",    StringType(),  True),
    StructField("dest",      StringType(),  True),
    StructField("air_time",  IntegerType(), True),
    StructField("distance",  IntegerType(), True),
    StructField("hour",      IntegerType(), True),
    StructField("minute",    IntegerType(), True),
])

In [8]:
df_airports = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_airports)
                  .load("../Data/airports.csv"))

df_planes = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_planes)
                  .load("../Data/planes.csv"))

df_flights = (spark.getOrCreate().read
                  .format("csv")
                  .option("header", "true")
                  .schema(schema_flights)
                  .load("../Data/flights.csv"))

In [11]:
# Expressoes regulares comuns
REGEX_ALPHA    = r'[a-zA-Z]+'
REGEX_INTEGER  = r'[0-9]+'
REGEX_FLOAT    = r'[0-9]+\.[0-9]+'
REGEX_ALPHANUM = r'[0-9a-zA-Z]+'
REGEX_EMPTY_STR= r'[\t ]+$'
REGEX_SPECIAL  = r'[!@#$%&*\(\)_]+'
REGEX_NNUMBER  = r'^N[1-9][0-9]{2,3}([ABCDEFGHJKLMNOPQRSTUVXWYZ]{1,2})'
REGEX_NNUMBER_INVALID = r'(N0.*$)|(.*[IO].*)'
REGEX_TIME_FMT = r'^(([0-1]?[0-9])|(2[0-3]))([0-5][0-9])$'

In [13]:
# Funcoes auxiiliares
def split_csv(line):
    return tuple(map(lambda x: x.replace('"',''), line.split(",")))

def check_empty_column(col):
    return (F.col(col).isNull() | (F.col(col) == '') | F.col(col).rlike(REGEX_EMPTY_STR))

# Airports

# Pergunta 1

In [ ]:
df1 = df_airports.withColumn(
    'qa_faa', 
    when(
            (
                (col('faa').isNull()) |
                (col('faa') == "NA")
            ), "M"
        )
    .when(~F.col("faa").rlike("^(?=.*[0-9])(?=.*[a-zA-Z])(?=[A-Z0-9]).{3,5}$"), "F")
)

# Teste do resultado

In [13]:
df_airports.select('faa', 'qa_faa').where(col('qa_faa')=='F').show()

+---+------+
|faa|qa_faa|
+---+------+
+---+------+



In [15]:
df1.groupBy('qa_faa').count().show()

NameError: name 'df1' is not defined

### Pergunta 1 revisada 
- Utilizando o mesmo nome do Dataframe<br> 
- Utilizando função para realizar o tratamento

In [9]:
def check_empty_column(col):
    return (F.col(col).isNull() | (F.col(col) =='') | F.col(col).rlike(REGEX_EMPTY_STR))

In [12]:
df_airports = df_airports.withColumn('qa_faa',(
        F.when(check_empty_column('faa'),'M')
         .when(
             (~F.length(F.col('faa')).between(3,5)  |


In [14]:
df_airports.select('faa', 'qa_faa').where(col('qa_faa')=='F').show()

+---+------+
|faa|qa_faa|
+---+------+
+---+------+



# Pergunta 2

In [ ]:
df2 = df1.withColumn(
    'qa_name', 
    when(
            (
                (col('name').isNull()) |
                (col('name') == "NA")
            ), "M"
        )
)

## Pergunta 2 revisada 
- Utilizando função para tratar valor nulo e na

In [16]:
# Usando dataframe
df_airports = df_airports.withColumn('qa_name', F.when(check_empty_column('name'), 'M'))
df_airports.show(5)

+---+--------------------+---------+---------+----+---+---+------+-------+
|faa|                name|      lat|      lon| alt| tz|dst|qa_faa|qa_name|
+---+--------------------+---------+---------+----+---+---+------+-------+
|04G|   Lansdowne Airport|41.130474|-80.61958|1044| -5|  A|  null|   null|
|06A|Moton Field Munic...| 32.46057|-85.68003| 264| -5|  A|  null|   null|
|06C| Schaumburg Regional| 41.98934|-88.10124| 801| -6|  A|  null|   null|
|06N|     Randall Airport| 41.43191|-74.39156| 523| -5|  A|  null|   null|
|09J|Jekyll Island Air...|31.074472|-81.42778|  11| -4|  A|  null|   null|
+---+--------------------+---------+---------+----+---+---+------+-------+
only showing top 5 rows



# Teste do resultado

In [19]:
df_airports.groupBy('qa_name').count().show()

+-------+-----+
|qa_name|count|
+-------+-----+
|   null| 1394|
|      M|    3|
+-------+-----+



In [20]:
df_airports.select('name', 'qa_name').where(col('qa_name')=='M').show()

+--------------------+-------+
|                name|qa_name|
+--------------------+-------+
|          Immokalee |      M|
|Montgomery Region...|      M|
|Chicago Rockford ...|      M|
+--------------------+-------+



# Pergunta 3

In [ ]:
df3 = df2.withColumn(
    "qa_lat",
    when(~col("lat").between(-180, 180), "I")
    .when(
            (
                (col('lat').isNull()) |
                (col('lat') == "NA")
            ), "M"
        )
    .when(col("lat").rlike(REGEX_ALPHA), "A"))

### pergunta 3 revisada
- O código está igual, só a formatação que está direferente da minha. 

In [21]:
# Usando Dataframe
df_airports = df_airports.withColumn('qa_lat',(
      F.when(check_empty_column('lat'), 'M')
       .when(F.col('lat').rlike(REGEX_ALPHA),'A')
       .when(F.col('lat').between(-180,180),'I')))

df_airports.show(5)

+---+--------------------+---------+---------+----+---+---+------+-------+------+
|faa|                name|      lat|      lon| alt| tz|dst|qa_faa|qa_name|qa_lat|
+---+--------------------+---------+---------+----+---+---+------+-------+------+
|04G|   Lansdowne Airport|41.130474|-80.61958|1044| -5|  A|  null|   null|     I|
|06A|Moton Field Munic...| 32.46057|-85.68003| 264| -5|  A|  null|   null|     I|
|06C| Schaumburg Regional| 41.98934|-88.10124| 801| -6|  A|  null|   null|     I|
|06N|     Randall Airport| 41.43191|-74.39156| 523| -5|  A|  null|   null|     I|
|09J|Jekyll Island Air...|31.074472|-81.42778|  11| -4|  A|  null|   null|     I|
+---+--------------------+---------+---------+----+---+---+------+-------+------+
only showing top 5 rows



# Teste do resultado

In [22]:
df_airports.groupBy('qa_lat').count().show()

+------+-----+
|qa_lat|count|
+------+-----+
|     I| 1397|
+------+-----+



# Pergunta 4

In [ ]:
df4 = df3.withColumn(
    "qa_lon",
    when(~col("lon").between(-180, 180), "I")
    .when(
            (
                (col('lon').isNull()) |
                (col('lon') == "NA")
            ), "M"
        )
    .when(col("lon").rlike(REGEX_ALPHA), "A"))

# Pergunta 4 revisada
- Código igual apenas a formatação diferente.

In [24]:
# Usando dataframe
df_airports = df_airports.withColumn("qa_lon", (
    F.when(check_empty_column("lon"), "M")
     .when(~F.col('lon').rlike(REGEX_FLOAT),"A")
     .when(~F.col('lon').between(-180,180),"I")))

df_airports.show(5)

+---+--------------------+---------+---------+----+---+---+------+-------+------+------+
|faa|                name|      lat|      lon| alt| tz|dst|qa_faa|qa_name|qa_lat|qa_lon|
+---+--------------------+---------+---------+----+---+---+------+-------+------+------+
|04G|   Lansdowne Airport|41.130474|-80.61958|1044| -5|  A|  null|   null|     I|  null|
|06A|Moton Field Munic...| 32.46057|-85.68003| 264| -5|  A|  null|   null|     I|  null|
|06C| Schaumburg Regional| 41.98934|-88.10124| 801| -6|  A|  null|   null|     I|  null|
|06N|     Randall Airport| 41.43191|-74.39156| 523| -5|  A|  null|   null|     I|  null|
|09J|Jekyll Island Air...|31.074472|-81.42778|  11| -4|  A|  null|   null|     I|  null|
+---+--------------------+---------+---------+----+---+---+------+-------+------+------+
only showing top 5 rows



# Teste do resultado

In [25]:
df_airports.groupBy('qa_lon').count().show()

+------+-----+
|qa_lon|count|
+------+-----+
|  null| 1397|
+------+-----+



# Pergunta 5

In [ ]:
df5 = df4.withColumn(
    "qa_alt",
    when(col("alt") < 0, "I")
    .when(
            (
                (col('alt').isNull()) |
                (col('alt') == "NA")
            ), "M"
          )
    .when(~col("alt").rlike("[0-9]+"), "A")
)

### Pergunta 5 revisada
- código o mesmo apenas a formatação diferente.

In [26]:
# Usando dataframe
df_airports = df_airports.withColumn('qa_alt', (
            F.when(check_empty_column('alt'), 'M')
             .when(~F.col('alt').rlike(REGEX_INTEGER), 'A')             
             .when(F.col('alt') < 0, 'I')))

df_airports.show(5)

+---+--------------------+---------+---------+----+---+---+------+-------+------+------+------+
|faa|                name|      lat|      lon| alt| tz|dst|qa_faa|qa_name|qa_lat|qa_lon|qa_alt|
+---+--------------------+---------+---------+----+---+---+------+-------+------+------+------+
|04G|   Lansdowne Airport|41.130474|-80.61958|1044| -5|  A|  null|   null|     I|  null|  null|
|06A|Moton Field Munic...| 32.46057|-85.68003| 264| -5|  A|  null|   null|     I|  null|  null|
|06C| Schaumburg Regional| 41.98934|-88.10124| 801| -6|  A|  null|   null|     I|  null|  null|
|06N|     Randall Airport| 41.43191|-74.39156| 523| -5|  A|  null|   null|     I|  null|  null|
|09J|Jekyll Island Air...|31.074472|-81.42778|  11| -4|  A|  null|   null|     I|  null|  null|
+---+--------------------+---------+---------+----+---+---+------+-------+------+------+------+
only showing top 5 rows



# Teste do resultado

In [27]:
df_airports.groupBy(col('qa_alt')).count().show()

+------+-----+
|qa_alt|count|
+------+-----+
|  null| 1395|
|     I|    2|
+------+-----+



# Pergunta 6

In [ ]:
df6 = df5.withColumn(
    "qa_tz",
    when(
            (
                (col('tz').isNull()) |
                (col('tz') == "NA")
            ), "M"
        )
    .when(~col('tz').between(-11, 14), "I")
    .when(~F.col('tz').rlike(REGEX_INTEGER + '$'), 'A')
)


#  Pergunta 6 revisada
- código o mesmo formatacao diferente

In [29]:
# Usando dataframe
df_airports = df_airports.withColumn('qa_tz', (
            F.when(check_empty_column('tz'), 'M')
             .when(~F.col('tz').rlike(REGEX_INTEGER + '$'), 'A')         
             .when(~F.col('tz').between(-11, 14), 'I')))

df_airports.show(5)

+---+--------------------+---------+---------+----+---+---+------+-------+------+------+------+-----+
|faa|                name|      lat|      lon| alt| tz|dst|qa_faa|qa_name|qa_lat|qa_lon|qa_alt|qa_tz|
+---+--------------------+---------+---------+----+---+---+------+-------+------+------+------+-----+
|04G|   Lansdowne Airport|41.130474|-80.61958|1044| -5|  A|  null|   null|     I|  null|  null| null|
|06A|Moton Field Munic...| 32.46057|-85.68003| 264| -5|  A|  null|   null|     I|  null|  null| null|
|06C| Schaumburg Regional| 41.98934|-88.10124| 801| -6|  A|  null|   null|     I|  null|  null| null|
|06N|     Randall Airport| 41.43191|-74.39156| 523| -5|  A|  null|   null|     I|  null|  null| null|
|09J|Jekyll Island Air...|31.074472|-81.42778|  11| -4|  A|  null|   null|     I|  null|  null| null|
+---+--------------------+---------+---------+----+---+---+------+-------+------+------+------+-----+
only showing top 5 rows



# Teste do resultado

In [30]:
df_airports.groupBy(col('qa_tz')).count().show()

+-----+-----+
|qa_tz|count|
+-----+-----+
| null| 1397|
+-----+-----+



# Pergunta 7

In [ ]:
DST_CATEGORIES = ['E', 'A', 'S', 'O', 'Z', 'N', 'U']

df7 = df6.withColumn(
    "qa_dst",
    when(
            (
                (col('dst').isNull()) |
                (col('dst') == "NA")
            ), "M"
        )
             .when(F.col('dst').rlike(REGEX_INTEGER), 'N')
             .when(~F.col('dst').isin(DST_CATEGORIES), 'C')
)

# Pergunta 7 revisada
- código o mesmo, formatação diferente.

In [33]:
DST_CATEGORIES = ['E', 'A', 'S', 'O', 'Z', 'N', 'U']

# Usando dataframe
df_airports = df_airports.withColumn('qa_dst', (
            F.when(check_empty_column('dst'), 'M')
             .when(F.col('dst').rlike(REGEX_INTEGER), 'N')
             .when(~F.col('dst').isin(DST_CATEGORIES), 'C')))

df_airports.show(5)

+---+--------------------+---------+---------+----+---+---+------+-------+------+------+------+-----+------+
|faa|                name|      lat|      lon| alt| tz|dst|qa_faa|qa_name|qa_lat|qa_lon|qa_alt|qa_tz|qa_dst|
+---+--------------------+---------+---------+----+---+---+------+-------+------+------+------+-----+------+
|04G|   Lansdowne Airport|41.130474|-80.61958|1044| -5|  A|  null|   null|     I|  null|  null| null|  null|
|06A|Moton Field Munic...| 32.46057|-85.68003| 264| -5|  A|  null|   null|     I|  null|  null| null|  null|
|06C| Schaumburg Regional| 41.98934|-88.10124| 801| -6|  A|  null|   null|     I|  null|  null| null|  null|
|06N|     Randall Airport| 41.43191|-74.39156| 523| -5|  A|  null|   null|     I|  null|  null| null|  null|
|09J|Jekyll Island Air...|31.074472|-81.42778|  11| -4|  A|  null|   null|     I|  null|  null| null|  null|
+---+--------------------+---------+---------+----+---+---+------+-------+------+------+------+-----+------+
only showing top 5 

# Teste do resultado

In [34]:
df_airports.groupBy(col('qa_dst')).count().show()

+------+-----+
|qa_dst|count|
+------+-----+
|  null| 1397|
+------+-----+



# Salvando o arquivo em parquet com as colunas qa e informações importantes

In [ ]:
df7.select(
    col('faa'), 
    col('name'), 
    col('qa_faa'), 
    col('qa_name'), 
    col('qa_lat'), 
    col('qa_lon'), 
    col('qa_alt'), 
    col('qa_tz'), 
    col('qa_dst')
).write.parquet(
    path = 'C:/Users/coskata/Downloads/Datasets/parquet/airports.parquet',
    mode = 'overwrite'
)

# Salvando de um modo diferente sem passar todas as colunas.

In [ ]:
(df_airports.select(df_airports.colRegex("`^qa_.*`"))
            .repartition(1) # coalesce
            .write.format("parquet")
            .mode('overwrite')
            .option("header", "true")
            .save("Datasets/airports_qa.parquet"))

# Salvando o arquivo em parquet

In [ ]:
df7.write.parquet(
path = 'C:/Users/coskata/Downloads/Datasets/parquet/airports.parquet',
mode = 'overwrite'
)

# Teste do resultado

In [ ]:
path = 'C:/Users/coskata/Downloads/Datasets/parquet/airports.parquet'
airports_parquet = spark.read.parquet(path)

In [ ]:
airports_parquet.toPandas()

# Planes

# Pergunta 1

In [ ]:
df1 = df_planes.withColumn('qa_tailnum', (
        F.when(check_empty_column('tailnum'), 'M')
         .when(~F.length(F.col('tailnum')).between(5, 6), 'S')
         .when(~F.col('tailnum').startswith('N'), 'FN')
         .when( F.col('tailnum').rlike(REGEX_NNUMBER_INVALID), 'FE')
         .when(~F.col('tailnum').rlike(REGEX_NNUMBER), 'F')))

# Teste do resultado

In [ ]:
df1.groupBy('qa_tailnum').count().show()

# Pergunta 2

In [ ]:
df2 = df1.withColumn(
    "qa_year",
    when(
            (
                (col('year').isNull()) |
                (col('year') == "NA")
            ), "M"
        )
    .when(col('year') < 1950, "I")
)

# Pergunta 2 revisada
- O código é o mesmo, alterando apenas a formatação. 


In [36]:
# Usando dataframe
df_planes = df_planes.withColumn('qa_year', (
        F.when(check_empty_column('year'), 'M')
         .when(F.col('year') < 1950, 'I')))

df_planes.show(5)

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_year|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   null|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   null|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   null|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   null|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   null|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+
only showing top 5 rows



# Teste do resultado

In [ ]:
df2.groupBy('qa_year').count().show()

# Pergunta 3

In [ ]:
df3 = df2.withColumn(
    "qa_type", 
    when(
            (
                (col('type').isNull()) |
                (col('type') == "NA")
            ), "M"
        )
    .when(~col("type").rlike("[Fixed wing multi engine,\
                               Fixed wing single engine,\
                               Rotorcraft]"), "C")
)

## Pergunta 3 revisada
- mesmo código de forma diferente 

In [37]:
# Usando dataframe
PLANE_TYPES = ["Fixed wing multi engine", "Fixed wing single engine", "Rotorcraft"]
df_planes = df_planes.withColumn('qa_type', (
        F.when(check_empty_column('type'), 'M')
         .when(~F.col('type').isin(PLANE_TYPES), 'C')))

df_planes.show(5)

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+-------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_year|qa_type|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+-------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   null|   null|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   null|   null|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   null|   null|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   null|   null|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   null|   null|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+-------+
only showing top 5 rows



# Teste do resultado

In [15]:
df3.groupBy('qa_type').count().show()

NameError: name 'df3' is not defined

# Pergunta 4

In [ ]:
df4 = df3.withColumn(
    "qa_manufacturer",
    when(
            (
                (col('manufacturer').isNull()) |
                (col('manufacturer') == "NA")
            ), "M"
        )
    .when(~col("manufacturer").rlike("[AIRBUS,\
                                        BOEING,\
                                        BOMBARDIER,\
                                        CESSNA,\
                                        EMBRAER,\
                                        SIKORSKY,\
                                        CANADAIR,\
                                        PIPER,\
                                        MCDONNELL DOUGLAS,\
                                        CIRRUS,\
                                        BELL,\
                                        KILDALL GARY,\
                                        LAMBERT RICHARD,\
                                        BARKER JACK,\
                                        ROBINSON HELICOPTER,\
                                        GULFSTREAM,\
                                        MARZ BARRY]"), "C"))

# Pergunta 4 revisada 
- De forma diferente utilizando expressao lambda e rlike

In [39]:



# Usando dataframe
PLANES_MANUFACTURERS = ["AIRBUS","BOEING","BOMBARDIER","CESSNA","EMBRAER","SIKORSKY","CANADAIR","PIPER","MCDONNELL DOUGLAS","CIRRUS","BELL","KILDALL GARY","LAMBERT RICHARD","BARKER JACK","ROBINSON HELICOPTER","GULFSTREAM","MARZ BARRY"]
REGEX_MANUFACTURERS = r'|'.join(map(lambda x : f".*({x}).*", PLANES_MANUFACTURERS))

df_planes = df_planes.withColumn('qa_manufacturer', (
        F.when(check_empty_column('manufacturer'), 'M')
         .when(~F.col('manufacturer').rlike(REGEX_MANUFACTURERS), 'C')))

df_planes.show(3,vertical=True)

-RECORD 0-------------------------------
 tailnum         | N102UW               
 year            | 1998                 
 type            | Fixed wing multi ... 
 manufacturer    | AIRBUS INDUSTRIE     
 model           | A320-214             
 engines         | 2                    
 seats           | 182                  
 speed           | null                 
 engine          | Turbo-fan            
 qa_year         | null                 
 qa_type         | null                 
 qa_manufacturer | null                 
-RECORD 1-------------------------------
 tailnum         | N103US               
 year            | 1999                 
 type            | Fixed wing multi ... 
 manufacturer    | AIRBUS INDUSTRIE     
 model           | A320-214             
 engines         | 2                    
 seats           | 182                  
 speed           | null                 
 engine          | Turbo-fan            
 qa_year         | null                 
 qa_type        

# Teste do resultado

In [ ]:
df4.groupBy('qa_manufacturer').count().show()

# Pergunta 5

In [ ]:
df5 = df4.withColumn(
    "qa_model", 
    when(
            (
                (col('model').isNull()) |
                (col('model') == "NA")
            ), "M"
        )
    .when(
            (
                (col('manufacturer') == "AIRBUS") & (~col('model').startswith("A")) |
                (col('manufacturer') == "BOEING") & (~col('model').startswith("7")) |
                (col('manufacturer') == "BOMBARDIER") & (~col('model').startswith("CL")) |
                (col('manufacturer') == "MCDONNELL DOUGLAS") & (~col('model').startswith("MD") | col('model').startswith("DC"))
            ), "F"
        )
)

# Pergunta 5 revisada
- Mesmo código, formatação diferente



In [40]:
# Usando dataframe
df_planes = df_planes.withColumn('qa_model', (
        F.when(check_empty_column('model'), 'M')
         .when(
             ((F.col('manufacturer').rlike(r'.*AIRBUS.*') & ~F.col('model').startswith('A')) |
              (F.col('manufacturer').rlike(r'.*BOEING.*') & ~F.col('model').startswith('7')) |
              (F.col('manufacturer').rlike(r'.*(BOMBARDIER|CANADAIR).*') & ~F.col('model').startswith('CL')) |
              (F.col('manufacturer').rlike(r'.*MCDONNELL DOUGLAS.*') &  ~(F.col('model').startswith('MD') | F.col('model').startswith('DC')) )), 'F')))

df_planes.show(5)

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+-------+---------------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|qa_year|qa_type|qa_manufacturer|qa_model|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+-------+-------+---------------+--------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   null|   null|           null|    null|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   null|   null|           null|    null|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   null|   null|           null|    null|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|   null|   null|           null|    null|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|

# Teste do resultado

In [41]:
df_planes.groupBy('qa_model').count().show()

+--------+-----+
|qa_model|count|
+--------+-----+
|       F|   15|
|    null| 2613|
+--------+-----+



# Pergunta 6

In [ ]:
df6 = df5.withColumn(
    "qa_engines",
    when(
            (
                (col('engines').isNull()) |
                (col('engines') == "NA")
            ), "M"
        )
    .when(~col('engines').between(1,4), "I")
    .when(col('engines').rlike("^[0-9A-Z^]*$"), "A")
)

### Pergunta 6 revisada
- Mesmo código, formatação diferente

In [42]:
df_planes = df_planes.withColumn('qa_engines', (
        F.when(check_empty_column('engines'), 'M')
         .when(F.col("engines").between(1,4),'I')
         .when(F.col("engines").rlike(REGEX_ALPHA),'A')
))






df_planes.show(2,vertical=True)

-RECORD 0-------------------------------
 tailnum         | N102UW               
 year            | 1998                 
 type            | Fixed wing multi ... 
 manufacturer    | AIRBUS INDUSTRIE     
 model           | A320-214             
 engines         | 2                    
 seats           | 182                  
 speed           | null                 
 engine          | Turbo-fan            
 qa_year         | null                 
 qa_type         | null                 
 qa_manufacturer | null                 
 qa_model        | null                 
 qa_engines      | I                    
-RECORD 1-------------------------------
 tailnum         | N103US               
 year            | 1999                 
 type            | Fixed wing multi ... 
 manufacturer    | AIRBUS INDUSTRIE     
 model           | A320-214             
 engines         | 2                    
 seats           | 182                  
 speed           | null                 
 engine         

# Teste do resultado

In [43]:
df_planes.groupBy('qa_engines').count().show()

+----------+-----+
|qa_engines|count|
+----------+-----+
|         I| 2628|
+----------+-----+



# Pergunta 7

In [ ]:
df7 = df6.withColumn(
    "qa_seats",
    when(
            (
                (col('seats').isNull()) |
                (col('seats') == "NA")
            ), "M"
        )
    .when(~col('seats').between(2,500), "I")
    .when(col('seats').rlike("^[0-9A-Z^]*$"), "A")
)

# Teste do resultado

In [ ]:
df7.groupBy('qa_seats').count().show()

# Pergunta 8

In [ ]:
df8 = df7.withColumn(
    "qa_speed",
    when(
            (
                (col('speed').isNull()) |
                (col('speed') == "NA")
            ), "M"
        )
    .when(~col('speed').between(50,150), "I")
    .when(col('speed').rlike("^[0-9A-Z^]*$"), "A")
)

# pergunta 8 revisada
- Mesmo codigo, formatacao diferente

In [46]:
df_planes = df_planes.withColumn('qa_speed', (
        F.when(check_empty_column('speed'), 'M')
         .when(F.col('speed').between(50, 150), 'I')
         .when(F.col('speed').rlike(REGEX_ALPHANUM), 'A')
))

df_planes.show(2,vertical=True)

-RECORD 0-------------------------------
 tailnum         | N102UW               
 year            | 1998                 
 type            | Fixed wing multi ... 
 manufacturer    | AIRBUS INDUSTRIE     
 model           | A320-214             
 engines         | 2                    
 seats           | 182                  
 speed           | null                 
 engine          | Turbo-fan            
 qa_year         | null                 
 qa_type         | null                 
 qa_manufacturer | null                 
 qa_model        | null                 
 qa_engines      | I                    
 qa_seats        | I                    
 qa_speed        | M                    
-RECORD 1-------------------------------
 tailnum         | N103US               
 year            | 1999                 
 type            | Fixed wing multi ... 
 manufacturer    | AIRBUS INDUSTRIE     
 model           | A320-214             
 engines         | 2                    
 seats          

# Teste do resultado

In [47]:
df_planes.groupBy('qa_speed').count().show()

+--------+-----+
|qa_speed|count|
+--------+-----+
|       M| 2622|
|       I|    6|
+--------+-----+



# Pergunta 9

In [ ]:
df9 = df8.withColumn(
    "qa_engine",
    when(
            (
                (col('engine').isNull()) |
                (col('engine') == "NA")
            ), "M"
        )
    .when(
            (
                (~col('engine').rlike("Turbo-fan")) &
                (~col('engine').rlike("Turbo-jet")) &
                (~col('engine').rlike("Turbo-prop")) &
                (~col('engine').rlike("Turbo-shaft")) &
                (~col('engine').rlike("4 CycleY"))
            ), "C"
         )
)

# Pergunta 9 revisada
- Mesmo código formatação diferente

In [48]:
nome = ["Turbo-fan","Turbo-jet","Turbo-prop","Turbo-shaft","4 Cycle"]

df_planes = df_planes.withColumn('qa_engine', (
        F.when(check_empty_column('engine'), 'M')
         .when(~F.col('engine').isin(nome), 'C')))

df_planes.show(2,vertical=True)

-RECORD 0-------------------------------
 tailnum         | N102UW               
 year            | 1998                 
 type            | Fixed wing multi ... 
 manufacturer    | AIRBUS INDUSTRIE     
 model           | A320-214             
 engines         | 2                    
 seats           | 182                  
 speed           | null                 
 engine          | Turbo-fan            
 qa_year         | null                 
 qa_type         | null                 
 qa_manufacturer | null                 
 qa_model        | null                 
 qa_engines      | I                    
 qa_seats        | I                    
 qa_speed        | M                    
 qa_engine       | null                 
-RECORD 1-------------------------------
 tailnum         | N103US               
 year            | 1999                 
 type            | Fixed wing multi ... 
 manufacturer    | AIRBUS INDUSTRIE     
 model           | A320-214             
 engines        

# Teste do resultado

In [49]:
df_planes.groupBy('qa_engine').count().show()

+---------+-----+
|qa_engine|count|
+---------+-----+
|     null| 2618|
|        C|   10|
+---------+-----+



# Salvando o arquivo em parquet com as colunas qa e informações importantes

In [ ]:
df9.select(
    col('tailnum'), 
    col('qa_tailnum'),
    col('qa_year'), 
    col('qa_type'), 
    col('qa_manufacturer'), 
    col('qa_model'), 
    col('qa_engines'), 
    col('qa_seats'), 
    col('qa_speed'), 
    col('qa_engine')

).write.parquet(
    path = 'C:/Users/coskata/Downloads/Datasets/parquet/planes.parquet',
    mode = 'overwrite'
)

# Salvando de outra maneira sem passar todas as colunas


In [ ]:
(df_airports.select(df_airports.colRegex("`^qa_.*`"))
            .repartition(1) # coalesce
            .write.format("parquet")
            .mode('overwrite')
            .option("header", "true")
            .save("Datasets/airports_qa.parquet"))

# Salvando o arquivo em parquet

In [ ]:
df9.write.parquet(
path = 'C:/Users/coskata/Downloads/Datasets/parquet/planes.parquet',
mode = 'overwrite'
)

# Teste do resultado

In [ ]:
path = 'C:/Users/coskata/Downloads/Datasets/parquet/planes.parquet'
planes_parquet = spark.read.parquet(path)

In [ ]:
planes_parquet.toPandas()

# Airports

# Pergunta 1

In [56]:
df1 = df_airports.withColumn(
    "qa_year_month_day", 
    when(
            (
                (col('year').isNull()) |
                (col('year') == "NA")
            ), "MY"
        )
    .when(
            (
                (col('month').isNull()) |
                (col('month') == "NA")
            ), "MM"
        )
    .when(
            (
                (col('day').isNull()) |
                (col('day') == "NA")
            ), "MD"
        )
    .when(col("year") < 1950, "IY")
    .when(~col("month").between(1,12), "IM")
    .when(
            (~col('day').between(1,31)) | 
            (col("month") == 2) & 
            (~col('day').between(1,29)), "ID"
        )
   )

AnalysisException: cannot resolve 'year' given input columns: [alt, dst, faa, lat, lon, name, qa_alt, qa_dst, qa_faa, qa_lat, qa_lon, qa_name, qa_tz, tz];
'Project [faa#0, name#1, lat#2, lon#3, alt#4, tz#5, dst#6, qa_faa#64, qa_name#97, qa_lat#188, qa_lon#285, qa_alt#376, qa_tz#487, qa_dst#590, CASE WHEN (isnull('year) OR ('year = NA)) THEN MY WHEN (isnull('month) OR ('month = NA)) THEN MM WHEN (isnull('day) OR ('day = NA)) THEN MD WHEN ('year < 1950) THEN IY WHEN NOT (('month >= 1) AND ('month <= 12)) THEN IM WHEN (NOT (('day >= 1) AND ('day <= 31)) OR (('month = 2) AND NOT (('day >= 1) AND ('day <= 29)))) THEN ID END AS qa_year_month_day#1628]
+- Project [faa#0, name#1, lat#2, lon#3, alt#4, tz#5, dst#6, qa_faa#64, qa_name#97, qa_lat#188, qa_lon#285, qa_alt#376, qa_tz#487, CASE WHEN ((isnull(dst#6) OR (dst#6 = )) OR RLIKE(dst#6, [\t ]+$)) THEN M WHEN RLIKE(dst#6, [0-9]+) THEN N WHEN NOT dst#6 IN (E,A,S,O,Z,N,U) THEN C END AS qa_dst#590]
   +- Project [faa#0, name#1, lat#2, lon#3, alt#

# Pergunta 1 revisada
-- mesmo código, formatado de forma diferente

In [59]:
df_flights = df_flights.withColumn('qa_year_month_day',(
        F.when(check_empty_column('year'),'MY')
         .when(check_empty_column('month'),'MM')
         .when(check_empty_column('day'),'MD')
         .when((F.col('year') > 1950),'IY')
         .when((F.col('month').between(1,12)),'IM')
         .when((F.col('day').between(1,31)) &(F.col('day') =='2'),'ID')
         ))

# Teste do resultado

In [60]:
df_flights.groupBy('qa_year_month_day').count().show()

+-----------------+-----+
|qa_year_month_day|count|
+-----------------+-----+
|               IY|10000|
+-----------------+-----+



# Pergunta 2

In [ ]:
df2 = df1.withColumn(
    "qa_hour_minute",
    when(
            (
                (col('hour').isNull()) |
                (col('hour') == "NA")
            ), "MH"
        )
    .when(
            (
                (col('minute').isNull()) |
                (col('minute') == "NA")
            ), "MM"
        )
    .when(~col('hour').between(0,24), "IH")
    .when(~col('minute').between(0, 59), "IM")
)

# Teste do resultado

In [ ]:
df2.groupBy('qa_hour_minute').count().show()

# Pergunta 3

In [ ]:
# Usando dataframe
df3 = df2.withColumn('qa_dep_arr_time', (
        F.when(check_empty_column('dep_time'), 'MD')
         .when(check_empty_column('arr_time'), 'MA')
         .when(F.col('dep_time').rlike(REGEX_TIME_FMT), 'FD')
         .when(F.col('arr_time').rlike(REGEX_TIME_FMT), 'FA')))

# Teste do resultado

In [ ]:
df3.groupBy('qa_dep_arr_time').count().show()

# Pergunta 4

In [ ]:
df4 = df3.withColumn(
    "qa_dep_arr_delay",
    when(
            (
                (col('dep_delay').isNull()) |
                (col('dep_delay') == "NA")
            ), "MD"
        )
    .when(
            (
                (col('arr_delay').isNull()) |
                (col('arr_delay') == "NA")
            ), "MA"
        )
)

# Pergunta 4 revisada
- mesmo código formatacao diferente

In [63]:
df_flights = df_flights.withColumn('qa_dep_arr_delay', (
        F.when((F.col('dep_delay').isNull()) |
               (F.col('dep_delay') == ''), 'MD')
        .when((F.col('arr_delay').isNull())  |
              (F.col('arr_delay') == ''), 'MA')
        
))

# Teste do resultado

In [64]:
df_flights.groupBy('qa_dep_arr_delay').count().show()

+----------------+-----+
|qa_dep_arr_delay|count|
+----------------+-----+
|            null| 9925|
|              MD|   48|
|              MA|   27|
+----------------+-----+



# Pergunta 5

In [ ]:
df5 = df4.withColumn(
    "qa_carrier", 
    when(
            (
                (col('carrier').isNull()) |
                (col('carrier') == "NA")
            ), "M"
        )
    .when(~col('carrier').rlike("{2}(^[0-9][A-Z]|^[A-Z][0-9])$"), "F")
)

# Teste do resultado

In [ ]:
df5.groupBy('qa_carrier').count().show()

# Pergunta 6

In [ ]:
df6 = df5.withColumn('qa_tailnum', (
        F.when(check_empty_column('tailnum'), 'M')
         .when(~F.length(F.col('tailnum')).between(5, 6), 'S')
         .when(~F.col('tailnum').startswith('N'), 'FN')
         .when( F.col('tailnum').rlike(REGEX_NNUMBER_INVALID), 'FE')
         .when(~F.col('tailnum').rlike(REGEX_NNUMBER), 'F')))

# Teste do resultado

In [ ]:
df6.groupBy('qa_tailnum').count().show()

# Pergunta 7

In [ ]:
df7 = df6.withColumn(
    "qa_flight", 
    when(
            (
                (col('flight').isNull()) |
                (col('flight') == "NA")
            ), "M"
        )
    .when(~col("flight").rlike("[0-9]{4}$"), "F")
)

# Pergunta 7 Revisada
- mesmo código formatacao diferente

In [65]:
df_flights = df_flights.withColumn ('qa_flight',   (
    F.when(
         (F.col('flight').isNull()) | 
         (F.col('flight') == ''), 'M')
     .when(
         (~F.col('flight').rlike("[0-9]{4}")),'F')
    
))

# Teste do resultado

In [66]:
df_flights.groupBy(col('qa_flight')).count().show()

+---------+-----+
|qa_flight|count|
+---------+-----+
|        F| 6158|
|     null| 3842|
+---------+-----+



# Pergunta 8

In [ ]:
df8 = df7.withColumn(
    "qa_origin_dest", 
    when(
            (
                (col('origin').isNull()) |
                (col('origin') == "NA")
            ), "MO"
        )
    .when(
            (
                (col('dest').isNull()) |
                (col('dest') == "NA")
            ), "MD"
        )
    .when(
            (
                (col('origin').rlike("^[A-Z]*")) |
                (col('origin').rlike("^[0-9]*"))
            ), "FO"
        )
    .when(
            (
                (col('dest').rlike("^[A-Z]*")) |
                (col('dest').rlike("^[0-9]*"))
            ), "FD"
        )
)

### pergunta 8 revisada
- mesmo código formatacao diferente

In [ ]:
df_flights = df_flights.withColumn('qa_origin_dest', (
    F.when((F.col('origin').isNull()) | (F.col('origin') == ''), 'MO')
     .when((F.col('dest').isNull())| (F.col('dest') == ''), 'MD')
     .when((F.col('origin').rlike("[0-9a-zA-Z]{3}")),'FO')                            
     .when((F.col('dest').rlike("[0-9a-zA-Z]{3}")),'FD')
    
))

# Teste do resultado

In [ ]:
df8.groupBy('qa_origin_dest').count().show()

# Pergunta 9

In [ ]:
df9 = df8.withColumn(
    "qa_air_time", 
    when(
            (
                (col('air_time').isNull()) |
                (col('air_time') == "NA")
            ), "M"
        )
    .when(~col("air_time").between(20,500), "I")
)

# Teste do resultado

In [ ]:
df9.groupBy('qa_air_time').count().show()

# Pergunta 10

In [ ]:
df10 = df9.withColumn(
    "qa_distance", 
    when(
            (
                (col('distance').isNull()) |
                (col('distance') == "NA")
            ), "M"
        )
    .when(~col("distance").between(50,3000), "I")
)

# Teste do resultado

In [ ]:
df10.groupBy('qa_distance').count().show()

# Pergunta 11

In [ ]:
df11 = df10.withColumn('qa_distance_airtime', (
        F.when(check_empty_column('distance'), 'M')
         .when((F.col('air_time') >= F.col('distance') * 0.1 + 30), 'TL')
         .when((F.col('air_time') <= F.col('distance') * 0.1 + 10), 'TS')
         .otherwise('TR')))

# Teste do resultado

In [ ]:
df11.groupBy('qa_distance_airtime').count().show()

# Salvando o arquivo em parquet com as colunas qa e informações importantes

In [ ]:
df11.select(
    col('tailnum'), 
    col('flight'), 
    col('origin'), 
    col('dest'), 
    col('distance'), 
    col('qa_year_month_day'), 
    col('qa_hour_minute'), 
    col('qa_dep_arr_time'), 
    col('qa_dep_arr_delay'),
    col('qa_carrier'), 
    col('qa_tailnum'), 
    col('qa_flight'), 
    col('qa_origin_dest'), 
    col('qa_air_time'), 
    col('qa_distance_airtime')
).write.parquet(
    path = 'C:/Users/coskata/Downloads/Datasets/parquet/flights.parquet',
    mode = 'overwrite'
)

# Salvando o arquivo em parquet

In [ ]:
df11.write.parquet(
path = 'C:/Users/coskata/Downloads/Datasets/parquet/flights.parquet',
mode = 'overwrite'
)

# Teste do resultado

In [ ]:
path = 'C:/Users/coskata/Downloads/Datasets/parquet/flights.parquet'
flights_parquet = spark.read.parquet(path)

In [ ]:
flights_parquet.toPandas()